<a href="https://colab.research.google.com/github/Azhar-999-coder/conversational_bot/blob/master/Response_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import tensorflow as tf
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply, GRU
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.models import Model
import numpy as np
import os
import matplotlib.pyplot as plt

In [79]:
BATCH_SIZE = 100
n_a = 64
n_s = 64

In [80]:
num_words=30000
max_length=30
Tx = max_length
Ty = max_length

path = 'Data/'

dirlist = os.listdir(path)
human_sentences=[]
machine_sentences=[]
for File in dirlist:
    with open(path+"/"+File, 'r') as raw_lines:
        lineList = []
        while True:        
            line = raw_lines.readline()
            if not line:
                break
            lineList.append(line)

    for i in range(0, len(lineList)):
        if(i%2)==0:
            human_sentences.append(lineList[i])
        else:
            machine_sentences.append(lineList[i])    

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token='<OOV>')
tokenizer.fit_on_texts(human_sentences)
human_word_index = tokenizer.word_index
human_reverse_word_index = {a:b for (b,a) in human_word_index.items()}
tokenizerE=tokenizer

tokenizer2 = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token='<OOV>')
    
tokenizer2.fit_on_texts(machine_sentences)
machine_word_index = tokenizer2.word_index
machine_reverse_word_index = {a:b for (b,a) in machine_word_index.items()}

human_sequences = tokenizer.texts_to_sequences(human_sentences)
human_padded = tf.keras.preprocessing.sequence.pad_sequences(human_sequences, maxlen=max_length, truncating="post")

machine_sequences = tokenizer2.texts_to_sequences(machine_sentences)
machine_padded = tf.keras.preprocessing.sequence.pad_sequences(machine_sequences, maxlen=max_length, truncating="post")


X = human_padded
Y = machine_padded
human_vocab = human_word_index
reverse_human_vocab = human_reverse_word_index
machine_vocab = machine_word_index
reverse_machine_vocab = machine_reverse_word_index


In [81]:
#taking care of odd-even shit
if X.shape[0]>Y.shape[0]:
  X = np.delete(X, len(X)-1, axis=0)
elif X.shape[0]<Y.shape[0]:
  Y = Y.delete(Y, len(Y)-1, axis=0)

In [82]:
#creating dataset
dataset = tf.data.Dataset.from_tensor_slices((X,Y)).shuffle(BATCH_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [83]:
print(Y)

[[   0    0    0 ...   82 2661 2856]
 [   0    0    0 ... 3489 3999 2857]
 [   0    0    0 ...   69 2858   35]
 ...
 [   0    0    0 ...   18   17    5]
 [   0    0    0 ...  692   17   45]
 [   0    0    0 ...  332   73   39]]


In [84]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, batch_size=100, dim=max_length, shuffle=False):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.len_per_epoch = int(len(X)/self.batch_size)
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.len_per_epoch

    def __getitem__(self, index):
        
        p,q = self.__data_generation()

        return p, q

    def __data_generation(self):
        
        X = np.empty((self.batch_size, self.dim))
        y = np.empty((self.batch_size, self.dim))
        s0 = np.zeros((self.batch_size, n_s))
        c0 = np.zeros((self.batch_size, n_s))
        X, y = next(iter(dataset))
        y = tf.transpose(y)

        return [X,s0,c0], y

In [21]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip

--2020-06-26 14:00:00--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2020-06-26 14:00:00--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2020-06-26 14:00:00--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [22]:
!unzip glove.twitter.27B.zip

Archive:  glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


In [85]:
embeddings_index = {}
f = open('glove.twitter.27B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [86]:
embedding_dim=100
embedding_matrix = np.zeros((len(human_vocab)+1, embedding_dim))
for word, i in human_vocab.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [87]:
embeddingLayer = tf.keras.layers.Embedding((len(human_vocab)+1),embedding_dim, weights=[embedding_matrix], trainable=False, input_length=max_length)

In [88]:
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor = Dense(1, activation = "relu")
dotor = Dot(axes = 1)

In [89]:
def one_step_attention(a, s_prev):
    
    s_prev = repeator(s_prev)
    
    concat = concatenator([a, s_prev])
    
    e = densor(concat)
    
    alphas = tf.nn.softmax(e, axis=1)
    
    context = dotor([alphas, a])
    
    return context

In [90]:
post_activation_LSTM_cell = LSTM(n_s, return_state = True)

In [91]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    
    X = Input(shape=(Tx,))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []

    x = embeddingLayer(X)
    print("Shape of x after embedding:")
    print(x.shape)
    
    # Step 1: Define your pre-attention Bi-LSTM. Remember to use return_sequences=True. (≈ 1 line)
    a = Bidirectional(LSTM(n_a, return_sequences=True))(x)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        print(context.shape)
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        print(s.shape)
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        p = Dense(machine_vocab_size)(s)
        out = tf.nn.softmax(p, axis=-1)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    outputs = tf.convert_to_tensor(outputs)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model([X, s0, c0], outputs)
    
    return model

In [92]:
model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))
model.summary()


Shape of x after embedding:
(None, 30, 100)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
(None, 1, 128)
(None, 64)
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #

In [93]:
opt = tf.keras.optimizers.Adam()
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)

In [94]:
generator = DataGenerator()
model.fit_generator(generator=generator, epochs=5)

Epoch 1/5
500/500 [==============================] - 38s 76ms/step - loss: 0.8080
Epoch 2/5
500/500 [==============================] - 38s 76ms/step - loss: 0.7500
Epoch 3/5
500/500 [==============================] - 38s 76ms/step - loss: 0.7526
Epoch 4/5
500/500 [==============================] - 38s 76ms/step - loss: 0.7517
Epoch 5/5
500/500 [==============================] - 38s 76ms/step - loss: 0.7509


In [123]:
Examples = ['How are you?','What are you doing?','I see you are a man of culture as well']
sequenceE = tokenizer.texts_to_sequences(Examples)
X = tf.keras.preprocessing.sequence.pad_sequences(sequences=sequenceE, maxlen=max_length, truncating = "post")
print(X.shape)
s0 = np.zeros((len(Examples),n_s))
c0 = np.zeros((len(Examples),n_s))
prediction = model.predict([X,s0,c0])
perm = [1,0,2]
prediction = tf.transpose(prediction, perm=perm)
responses = []
prediction = np.array(prediction)

for i in range(0,prediction.shape[0]):
  temp = []
  for j in range(0, prediction.shape[1]):
    prediction[i][j][0]=0.000000000000000000001
    p = np.argmax(prediction[i][j])
    temp.append(p)
  responses.append(temp)
for i in range(0, len(responses)):
 responses[i] = [reverse_machine_vocab[int(o)] for o in responses[i]]
print(responses)


(3, 30)
[["on'", 'jenkins', 'pleased', 'swamp', "'mam", 'character', 'monsters', 'hyenas', 'monstruo', 'deserted', 'drinking', 'cuff', "seregay's", 'ieast', 'since', 'you', 'inherited', 'exactly', 'two', 'years', 'ago', 'today', 'she', 'and', 'i', 'i', 'what', 'you', 'you', 'you'], ["on'", 'jenkins', 'pleased', 'swamp', "'mam", 'character', 'monsters', 'hyenas', 'monstruo', 'deserted', 'drinking', 'cuff', "seregay's", 'ieast', 'since', 'you', 'inherited', 'exactly', 'two', 'years', 'ago', 'today', 'she', 'and', 'i', 'i', 'what', 'you', 'you', 'you'], ["on'", 'jenkins', 'pleased', 'swamp', "'mam", 'character', 'monsters', 'hyenas', 'monstruo', 'deserted', 'drinking', 'cuff', "seregay's", 'ieast', 'since', 'you', 'inherited', 'exactly', 'two', 'years', 'ago', 'today', 'she', 'and', 'i', 'i', 'what', 'you', 'you', 'you']]


In [ ]:
print(human_sentences)

['Presented by IM Pictures\n', 'In association with MVP Venture Capital and Cinema Service\n', 'My Sassy Girl\n', "We promised to meet here two years later, but she hasn't come yet.\n", 'Here we go.\n', 'One, two...\n', 'Hello?\n', "Sorry, I'm on my way.\n", "Yes, I'm coming.\n", 'Bye.\n', 'Here we go.\n', 'My parents wanted a daughter, so they raised me like one.\n', "I had to go to the women's public bath, too.\n", 'I thought my penis would get smaller and disappear.\n', 'First Half\n', "No, I'm a real man now.\n", 'Think clerical work in the army makes you a man?\n', 'Give me a break, asshole.\n', 'Hey!\n', 'Who are you kidding?\n', 'Anyway, welcome back home.\n', "When I see my type, I can't help it.\n", "Who's interrupting me?\n", 'Who is this?\n', '- Oh, mom...\n', "I'm leaving soon.\n", "It's my mom!\n", 'Make sure you pay a visit.\n', 'That long?\n', 'She says you resemble him.\n', 'Still there?\n', 'Plus, I hate when she rubs my face and kisses me.\n', "She'll introduce you to

In [114]:
T = [1,2,6,5,4,6,8,9]
prediction[0][0][0] = 0.000000000000000000000000000000000000000000000000000000000000000000000000000004
print(np.argmax(prediction[0][0]))

6198
